## Building user-based recommendation model for Amazon

### Analysis Task
##### - Exploratory Data Analysis:

###### Which movies have maximum views/ratings?
###### What is the average rating for each movie? Define the top 5 movies with the maximum ratings.
###### Define the top 5 movies with the least audience.

##### - Recommendation Model: Some of the movies hadn’t been watched and therefore, are not rated by the users. Netflix would like to take this as an opportunity and build a machine learning recommendation algorithm which provides the ratings for each of the users.

###### Divide the data into training and test data
###### Build a recommendation model on training data
###### Make predictions on the test data

In [4]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import surprise

In [5]:
df = pd.read_csv('../input/Amazon - Movies and TV Ratings.csv')

In [6]:
df.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.shape

(4848, 207)

In [8]:
df_org = df.copy()

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Movie1,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie2,1.0,5.000000,NaN,5.0,5.00,5.0,5.0,5.0
Movie3,1.0,2.000000,NaN,2.0,2.00,2.0,2.0,2.0
Movie4,2.0,5.000000,0.000000,5.0,5.00,5.0,5.0,5.0
Movie5,29.0,4.103448,1.496301,1.0,4.00,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...
Movie202,6.0,4.333333,1.632993,1.0,5.00,5.0,5.0,5.0
Movie203,1.0,3.000000,NaN,3.0,3.00,3.0,3.0,3.0
Movie204,8.0,4.375000,1.407886,1.0,4.75,5.0,5.0,5.0
Movie205,35.0,4.628571,0.910259,1.0,5.00,5.0,5.0,5.0


### Task 1  - Which movies have maximum views/ratings?

In [10]:
#Movie with highest views
df.describe().T['count'].sort_values(ascending=False)[:1].to_frame() #---Movie127

,count
Movie127,2313.0


In [11]:
#Movie with highest Ratings
df.drop('user_id',axis=1).sum().sort_values(ascending=False)[:1].to_frame()  #---Movie127

,0
Movie127,9511.0


### Task 2 - What is the average rating for each movie? Define the top 5 movies with the maximum ratings

In [12]:
df.drop('user_id',axis=1).mean().sort_values(ascending=False)[:5].to_frame()

,0
Movie1,5.0
Movie55,5.0
Movie131,5.0
Movie132,5.0
Movie133,5.0


### Task 3 - Define the top 5 movies with the least audience

In [13]:
df.describe().T['count'].sort_values(ascending=True)[:5].to_frame()

,count
Movie1,1.0
Movie71,1.0
Movie145,1.0
Movie69,1.0
Movie68,1.0


### Task 4 - Recommendation Model

In [14]:
from surprise import Reader
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise.model_selection import cross_validate

In [15]:
df_melt = df.melt(id_vars = df.columns[0],value_vars=df.columns[1:],var_name="Movies",value_name="Rating")

In [16]:
df_melt

,user_id,Movies,Rating
0,A3R5OBKS7OM2IR,Movie1,5.0
1,AH3QC2PC1VTGP,Movie1,NaN
2,A3LKP6WPMP9UKX,Movie1,NaN
3,AVIY68KEPQ5ZD,Movie1,NaN
4,A1CV1WROP5KTTW,Movie1,NaN
...,...,...,...
998683,A1IMQ9WMFYKWH5,Movie206,5.0
998684,A1KLIKPUF5E88I,Movie206,5.0
998685,A5HG6WFZLO10D,Movie206,5.0
998686,A3UU690TWXCG1X,Movie206,5.0


In [17]:
rd = Reader()
data = Dataset.load_from_df(df_melt.fillna(0),reader=rd)
data

In [18]:
trainset, testset = train_test_split(data,test_size=0.25)

In [19]:
#Using SVD (Singular Value Descomposition)
svd = SVD()
svd.fit(trainset)

In [20]:
pred = svd.test(testset)

In [21]:
accuracy.rmse(pred)

RMSE: 1.0259


1.0258854020220565

In [22]:
accuracy.mae(pred)

MAE:  1.0120


1.0120319242726175

In [23]:
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0260  1.0266  1.0256  1.0261  0.0004  
MAE (testset)     1.0120  1.0122  1.0118  1.0120  0.0002  
Fit time          86.29   85.28   85.04   85.54   0.54    
Test time         6.38    5.86    5.78    6.00    0.27    


{'test_rmse': array([1.02599135, 1.02659615, 1.02560436]),
 'test_mae': array([1.01204046, 1.0122414 , 1.01178283]),
 'fit_time': (86.287588596344, 85.28330588340759, 85.04037284851074),
 'test_time': (6.376158952713013, 5.85737156867981, 5.777690172195435)}

In [24]:
def repeat(ml_type,dframe):
    rd = Reader()
    data = Dataset.load_from_df(dframe,reader=rd)
    print(cross_validate(ml_type, data, measures = ['RMSE', 'MAE'], cv = 3, verbose = True))
    print("--"*15)
    usr_id = 'A3R5OBKS7OM2IR'
    mv = 'Movie1'
    r_u = 5.0
    print(ml_type.predict(usr_id,mv,r_ui = r_u,verbose=True))
    print("--"*15)


In [25]:
repeat(SVD(),df_melt.fillna(df_melt['Rating'].mean()))
#repeat(SVD(),df_melt.fillna(df_melt['Rating'].median()))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0881  0.0846  0.0849  0.0859  0.0016  
MAE (testset)     0.0097  0.0096  0.0097  0.0097  0.0000  
Fit time          84.78   84.60   84.76   84.71   0.08    
Test time         6.05    6.34    6.07    6.15    0.13    
{'test_rmse': array([0.08812511, 0.08455872, 0.08493285]), 'test_mae': array([0.00971393, 0.00964239, 0.00971664]), 'fit_time': (84.77509498596191, 84.6006224155426, 84.76175332069397), 'test_time': (6.0538084506988525, 6.33686637878418, 6.066934585571289)}
------------------------------
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.39   {'was_impossible': False}
user: A3R5OBKS7OM2IR item: Movie1     r_ui = 5.00   est = 4.39   {'was_impossible': False}
------------------------------


In [26]:
#trying grid search and find optimum hyperparameter value for n_factors
from surprise.model_selection import GridSearchCV

In [27]:
param_grid = {'n_epochs':[20,30],
             'lr_all':[0.005,0.001],
             'n_factors':[50,100]}

In [ ]:
gs = GridSearchCV(SVD,param_grid,measures=['rmse','mae'],cv=3)
data1 = Dataset.load_from_df(df_melt.fillna(df_melt['Rating'].mean()),reader=rd)
gs.fit(data1)

In [ ]:
gs.best_score

In [ ]:
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])